In [6]:
# Adapted from https://effectivepython.com/2015/03/10/consider-coroutines-to-run-many-functions-concurrently

from collections import namedtuple
from typing import Generator, Literal

ALIVE = '#'
EMPTY = '-'
TICK = object()
Query = namedtuple('Query', ['y', 'x'])
Transition = namedtuple('Transition', ['y', 'x', 'state'])

def get_neighbors(y: int, x: int):
    n_ = yield Query(y - 1, x    )
    ne = yield Query(y - 1, x + 1)
    e_ = yield Query(y    , x + 1)
    se = yield Query(y + 1, x + 1)
    s_ = yield Query(y + 1, x    )
    sw = yield Query(y + 1, x - 1)
    w_ = yield Query(y    , x - 1)
    nw = yield Query(y - 1, x - 1)
    return n_, ne, e_, se, s_, sw, w_, nw

def game_logic(state, neighbors: list):
    alive_count = neighbors.count(ALIVE)
    if state == ALIVE:
        if alive_count < 2:
            return EMPTY
        if alive_count > 3:
            return EMPTY
    else:
        if alive_count == 3:
            return ALIVE
    return state

def step_cell(y: int, x: int):
    state = yield Query(y, x)
    neighbors = yield from get_neighbors(y, x)
    next_state = game_logic(state, neighbors)
    yield Transition(y, x, next_state)

def update_board(height: int, width: int):
    while True:
        for y in range(height):
            for x in range(width):
                yield from step_cell(y, x)
        yield TICK

class Board(object):
    def __init__(self, height, width):
        self.height = height
        self.width = width
        self.board = [[EMPTY for _ in range(width)] for _ in range(height)]
    
    def query(self, y, x):
        return self.board[y % self.height][x % self.width]
    
    def transition(self, y, x, state):
        self.board[y][x] = state
    
    def __str__(self):
        return '\n'.join(''.join(row) for row in self.board)

def simulate_transition(grid: Board, update: Generator):
    progeny = Board(grid.height, grid.width)
    instr = next(update)
    while instr is not TICK:
        if isinstance(instr, Query):
            instr = update.send(grid.query(instr.y, instr.x))
        else:
            progeny.transition(instr.y, instr.x, instr.state)
            instr = next(update)
    return progeny

grid = Board(5, 9)
grid.transition(0, 3, ALIVE)
grid.transition(1, 4, ALIVE)
grid.transition(2, 2, ALIVE)
grid.transition(2, 3, ALIVE)
grid.transition(2, 4, ALIVE)
print(grid)

---#-----
----#----
--###----
---------
---------


In [8]:
class ColumnPrinter(object):
    def __init__(self, rows):
        self.col_count = 0
        self.rows = [''] * (rows + 1)

    def append(self, obj):
        board = obj.split()
        col_width = len(board[0])
        if not self.col_count:
            self.rows[0] = str(self.col_count).center(col_width)
        else:
            self.rows[0] += ' | ' + str(self.col_count).center(col_width)
        for i, obj_row in enumerate(board):
            if not self.col_count:
                self.rows[i + 1] = obj_row
            else:
                self.rows[i + 1] += ' | ' + obj_row
        self.col_count += 1

    def __str__(self):
        return '\n'.join(self.rows)

columns = ColumnPrinter(5)
sim = update_board(grid.height, grid.width)
for i in range(10):
    columns.append(str(grid))
    grid = simulate_transition(grid, sim)
print(columns)

    0     |     1     |     2     |     3     |     4     |     5     |     6     |     7     |     8     |     9    
-#------- | --------- | --------- | --------- | --------- | --------- | --------- | --------- | --------- | ---#-----
--#------ | #-#------ | --#------ | -#------- | --#------ | --------- | --------- | --------- | --------- | ---------
###------ | -##------ | #-#------ | --##----- | ---#----- | -#-#----- | ---#----- | --#------ | ---#----- | ---------
--------- | -#------- | -##------ | -##------ | -###----- | --##----- | -#-#----- | ---##---- | ----#---- | --#-#----
--------- | --------- | --------- | --------- | --------- | --#------ | --##----- | --##----- | --###---- | ---##----
